In [1]:
import spacy
from random import random
from bisect import bisect

# Lab 04 - Edit distance

### Implementacja algorytmu wyznaczania odległości edycyjnej wraz z odtworzonymi krokami

In [2]:
def delta(a, b):
    return 0 if a == b else 1

def edit_distance(x, y, delta, show_answer = False):
    m = len(x)
    n = len(y)
    table = [[0]*(n+1) for _ in range(m+1)]
    
    for i in range(m+1):
        table[i][0] = i
    
    for i in range(n+1):
        table[0][i] = i
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            table[i][j] = min(table[i-1][j] + 1, table[i][j-1]+1, table[i-1][j-1] + delta(x[i-1], y[j-1]))
    
    if show_answer:
        return([x + ' -> ' + y, '-'*(len(x) + len(y) + 4)] + get_solution(table, x, y, m, n))
    return table[m][n]
    
def get_solution(table, s1, s2, i, j):
    if table[i][j] == 0:
        return []
    
    if i == 0:
        step = s1 + ' -> ' + '|' + s2[j-1] + '|' + s1
        s1 = s2[j-1] + s1
        return [step] + get_solution(table, s1, s2, i, j-1)
    elif j == 0:
        step = s1 + '->' + s1[:-1] + '_'
        s1 = s1[:-1]
        return [step] + get_solution(table, s1, s2, i-1, j) 
    else:
        if table[i][j] == table[i-1][j] + 1:
            step = s1 + ' -> ' + s1[:i-1] + '_' + s1[i:]
            s1 = s1[:i-1] + s1[i:]
            return [step] + get_solution(table, s1, s2, i-1, j)
        elif table[i][j] == table[i][j-1] + 1:
            step = s1 + ' -> ' + s1[:i] + '|' + s2[j-1] + '|' + s1[i:]
            s1 = s1[:i] + s2[j-1] + s1[i:]
            return [step] + get_solution(table, s1, s2, i, j-1)

        if s1[i-1] == s2[j-1]:
            return get_solution(table, s1, s2, i-1, j-1)
        
        step = s1 + ' -> ' + s1[:i-1] + '*' + s2[j-1] + '*' + s1[i:]
        s1 = s1[:i-1] + s2[j-1] + s1[i:]
        return  [step] + get_solution(table, s1, s2, i-1, j-1)

In [3]:
for line in edit_distance('los', 'kloc', delta, True):
    print(line)

los -> kloc
-----------
los -> lo*c*
loc -> |k|loc


In [4]:
for line in edit_distance('Łódź', 'Lodz', delta, True):
    print(line)

Łódź -> Lodz
------------
Łódź -> Łód*z*
Łódz -> Ł*o*dz
Łodz -> *L*odz


In [5]:
for line in edit_distance('kwintesencja', 'quintessence', delta, True):
    print(line)

kwintesencja -> quintessence
----------------------------
kwintesencja -> kwintesencj_
kwintesencj -> kwintesenc*e*
kwintesence -> kwintes|s|ence
kwintessence -> k*u*intessence
kuintessence -> *q*uintessence


In [6]:
for line in edit_distance('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG', delta, True):
    print(line)

ATGAATCTTACCGCCTCG -> ATGAGGCTCTGGCCCCTG
----------------------------------------
ATGAATCTTACCGCCTCG -> ATGAATCTTACCGCCT_G
ATGAATCTTACCGCCTG -> ATGAATCTTACC_CCTG
ATGAATCTTACCCCTG -> ATGAATCTTA|G|CCCCTG
ATGAATCTTAGCCCCTG -> ATGAATCTT*G*GCCCCTG
ATGAATCTTGGCCCCTG -> ATGAATCT|C|TGGCCCCTG
ATGAATCTCTGGCCCCTG -> ATGAA*G*CTCTGGCCCCTG
ATGAAGCTCTGGCCCCTG -> ATGA*G*GCTCTGGCCCCTG


### Implementacja algorytmu znajdowania maksymalnego wspólnego podciągu

In [7]:
def longest_common_sequence(x, y):
    ranges = [len(y)]
    y_letters = list(y)
    for i in range(len(x)):
        positions = [j for j, l in enumerate(y_letters) if l == x[i]]
        positions.reverse()
        for p in positions:
            k = bisect(ranges, p)
            if k == bisect(ranges, p - 1):
                if k < len(ranges) - 1:
                    ranges[k] = p
                else:
                    ranges[k:k] = [p]
    return len(ranges) - 1

In [8]:
print('Najdłuższy wspólny podciąg słów los i kloc ma długość: {}'
     .format(longest_common_sequence('los', 'kloc')))

Najdłuższy wspólny podciąg słów los i kloc ma długość: 2


In [9]:
print('Najdłuższy wspólny podciąg słów Łódź i Lodz ma długość: {}'
     .format(longest_common_sequence('Łódź', 'Lodz')))

Najdłuższy wspólny podciąg słów Łódź i Lodz ma długość: 1


In [10]:
print('Najdłuższy wspólny podciąg słów kwintesencja i quintessence ma długość: {}'
     .format(longest_common_sequence('kwintesencja', 'quintessence')))

Najdłuższy wspólny podciąg słów kwintesencja i quintessence ma długość: 8


In [11]:
print('Najdłuższy wspólny podciąg słów ATGAATCTTACCGCCTCG i ATGAGGCTCTGGCCCCTG ma długość: {}'
     .format(longest_common_sequence('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG')))

Najdłuższy wspólny podciąg słów ATGAATCTTACCGCCTCG i ATGAGGCTCTGGCCCCTG ma długość: 13


### Podział tekstu na tokeny i usunięcie 3% tokenów

In [12]:
def text_with_removed_tokens(doc : spacy.tokens.Doc, tokens_to_delete : float):
    reduced_text = []
    
    for token in doc:
        if random() > tokens_to_delete :
            reduced_text += [token.text_with_ws]
        else:
            cnt = token.text_with_ws.count('\n')
            for _ in range(cnt):
                reduced_text += ['\n']
            
    return reduced_text

In [13]:
with open('romeo-i-julia-700.txt', mode='r', encoding='utf-8') as file:
    data = file.read()

    nlp = spacy.load("pl_core_news_sm")
    doc = nlp(data)
    reduced_text1 = text_with_removed_tokens(doc, 0.03)
    reduced_text2 = text_with_removed_tokens(doc, 0.03)
    
    
    with open('variant1.txt', mode='w', encoding='utf-8') as save_file:
        for token_text in reduced_text1:
            save_file.write(token_text)
    
    with open('variant2.txt', mode='w', encoding='utf-8') as save_file:
        for token_text in reduced_text2:
            save_file.write(token_text)
    
    lcs = longest_common_sequence(reduced_text1, reduced_text2)
    
    print('Ilość tokenów na które został podzielony tekst: {}'.format(len(doc)))
    print('Ilość tokenów usuniętych z pierwszej wersji podziału: {}'.format(len(doc) - len(reduced_text1)))
    print('Ilość tokenów usuniętych z drugiej wersji podziału: {}'.format(len(doc) - len(reduced_text2)))
    print('Najdłuższy wspólny podciąg tokenów: {}'.format(lcs))

Ilość tokenów na które został podzielony tekst: 2699
Ilość tokenów usuniętych z pierwszej wersji podziału: 51
Ilość tokenów usuniętych z drugiej wersji podziału: 53
Najdłuższy wspólny podciąg tokenów: 2564


In [14]:
def show_diff(x, y):
    L = [[0 for _ in range(len(y) + 1)] for _ in range(len(x) + 1)]
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            if x[i - 1] == y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    lines = []
    i = len(x) - 1
    j = len(y) - 1
    
    while i >= 0 and j >= 0:
        if x[i] == y[j]:
            i, j = i-1, j-1
        elif L[i][j-1] >= L[i-1][j]:
            lines.append(f"> |{j}| {y[j]}")
            j -= 1
        elif L[i][j-1] < L[i-1][j]:
            lines.append(f"< |{i}| {x[i]}")
            i -= 1
            
    while j >= 0:
        lines.append(f"> |{j}| {y[j]}")
        j -= 1
        
    while i >= 0:
        lines.append(f"< |{i}| {x[i]}")
        i -= 1
    lines.reverse()
    for line in lines:
        print(line)

In [15]:
with open('variant1.txt', mode='r', encoding='utf-8') as file:
    lines1 = file.read().splitlines()
with open('variant2.txt', mode='r', encoding='utf-8') as file:
    lines2 = file.read().splitlines()

show_diff(lines1, lines2)

< |2| Romeo Julia
> |2| Romeo i Julia
< |11|  * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
> |11|  * PARYS — młody Weroneńczyk szlachetnego rodukrewny księcia
< |12| * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
< |13|  * STARZEC — stryjeczny brat Kapuleta
< |14|  * ROMEO — syn Montekiego
> |12|  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
> |13|  * STARZEC — stryjeczny brat Kapuleta
> |14|  * ROMEO — Montekiego
< |18|  * LAURENTY ojciec franciszkanin
> |18| * LAURENTY — ojciec franciszkanin
< |19|  * JAN — brat z tegoż zgromadzenia
< |20|  * BALTAZAR — służący Romea
> |19|  * JAN — brat z tegoż zgromadzenia
> |20|  * BALTAZAR — Romea
< |25|  * PAŹ PARYSA
> |25| * PAŹ PARYSA
< |37| Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu Mantui.
> |37| Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego w Mantui.
< |45| Dwa rody, zacne jednako i sławne 
> |45| Dwa rody, zacne jednako 